## Stochastic gradient descent-based inference for dynamic network models with attractors
## This script analyzes the same Twitter congressional hashtag networks, but includes all nodes recorded during the study.

In [ ]:
from congress_utils import congress_clsna, preprocess_congress_2, make_ar_pair, member_dict, ClsnaModelCongress
from utils import visualize_membership, visualize
import numpy as np
import torch
import math
import matplotlib.pyplot as plt
from scipy.linalg import orthogonal_procrustes

In [ ]:
import time

In [ ]:
# Set global variables for the model
DIM = 2
SIGMA = 10
TAU = 1
PHI = 10
T=11

In [ ]:
# Preprocess data
import numpy as np
from scipy.spatial.distance import squareform
from sklearn.preprocessing import normalize
from scipy.sparse.csgraph import laplacian
import torch


#names [array(['biden','sanders',...]),...]
names = []
for i in range(1,12):
    names.append(list(np.load('name'+str(i)+'.npy')))
#Y [array Y1,...]
Y = []
for i in range(1,12):
    Y.append(np.load('Y'+str(i)+'.npy'))

real_name = list(np.load("real_name.npy"))
handle = list(np.load('handle.npy'))
party_id =(np.load('party_id.npy'))

party_id = list(np.where(party_id == 'D', 0, 1))

#membership [['D','R',...],...]
membership = []
for i in range(T):
    mem = []
    for name in names[i]:
        mem.append(party_id[handle.index(name)])
    membership.append(mem)

time_point = []
n_nodes = []
start_idx = 0
for arr in names:
    n_nodes.append(len(arr))
    time_point.append(np.arange(start_idx, start_idx + len(arr)))
    start_idx += len(arr)
# time_point: [array([0, 1, 2, 3]), array([4, 5, 6]), array([7, 8, 9])]

def process_arrays(t):
    index_pairs = []
    not_in_name1 = {0:[],1:[]}
    name1 = names[t-1]
    name2 = names[t]

    for i, name in enumerate(name2):
        if name in name1:
            j = name1.index(name)
            index_pairs.append((time_point[t-1][j],time_point[t][i]))
        else:
            party = party_id[handle.index(name)]
            if party not in not_in_name1:
                not_in_name1[party] = []
            not_in_name1[party].append(time_point[t][i])

    return index_pairs, not_in_name1

# # Example usage:
# handle = ["Alice", "Bob", "Charlie", "David", "Eva"]
# party_id = ["A", "B", "A", "B", "A"]
# name1 = ["Alice", "David", "Eva"]
# name2 = ["Eva","Charlie", "Bob","David"]

# index_pairs, grouped_by_party = process_arrays(handle, party_id, name1, name2)
# print("Index pairs:", index_pairs)
# print("Grouped by party:", grouped_by_party)

# #Output
# Index pairs: [(0, 2), (3, 1)]
# Grouped by party: {'A': ['Charlie'], 'B': ['Bob']}

#ar_pair, a T-1 element list of Nx2 array, each row of array is [prev,self]
ar_pair = []
#new_at_t, a T-1 element dict, key is time, each element is also dict, key is party name
new_at_t = {}
for i in range(1,T):
    index_pairs, grouped_by_party = process_arrays(i)
    ar_pair+=index_pairs
    new_at_t[i] = grouped_by_party


def group_by_party(t):
    parties = membership[t]
    party_groups = {}
    
    for i, party in enumerate(parties):
        if party not in party_groups:
            party_groups[party] = []

        party_groups[party].append(time_point[t][i])

    return party_groups

# names = ['John Doe', 'Jane Smith', 'Mike Johnson', 'Sara Brown']
# parties = ['Democrat', 'Republican', 'Democrat', 'Republican']

# result = group_by_party(names, parties)
# print(result)
# {'Democrat': [0, 2], 'Republican': [1, 3]}

member_at_t = {}
for i in range(T):
    member_at_t[i] = group_by_party(i)

device = 'cuda' if torch.cuda.is_available() else 'cpu'
# device = 'cpu'

import torch
def convert_to_tensor(data, device):
    return torch.tensor(data, dtype=torch.long, device=device)
def party_dict_to_tensor(party_dict,device):
    return {party:convert_to_tensor(indices,device) for party,indices in party_dict.items()}
    

ar_pair = convert_to_tensor(ar_pair,device)
member_at_t = {year: party_dict_to_tensor(p_dict,device) for year, p_dict in member_at_t.items()}
new_at_t = {year: party_dict_to_tensor(p_dict,device) for year, p_dict in new_at_t.items()}


def find_matching_indices(name1, name2):
    matching_indices = []
    for i, name in enumerate(name1):
        if name in name2:
            j = name2.index(name)
            matching_indices.append([i, j])
    return np.array(matching_indices)

def extract_subgraph_adjacency_matrix(Y_t1, matching_indices, Y_t2):
    size = Y_t2.shape[0]
    persist = np.zeros((size, size))
    i = matching_indices[:,0]
    j = matching_indices[:,1]
    persist[np.ix_(j, j)] = Y_t1[np.ix_(i, i)]
    return persist
def construct_persistence(t):
    matching_indices = find_matching_indices(names[t-1], names[t])
    persist = extract_subgraph_adjacency_matrix(Y[t-1], matching_indices, Y[t])
    return persist
persist = []
for i in range(1,T):
    persist.append(squareform(construct_persistence(i)))

y_t = squareform(Y[0])
y = [y_t]
Aw = []
Aw2 = []
Ab = []
s=np.array(membership[0])
for i in range(1,T):
    #normalized graph laplalcian by row
    Aw_t = squareform(y_t)*(np.outer(s, s))
    Aw_t = -2*normalize(laplacian(Aw_t), axis=1, norm='l1')
    Aw2_t = squareform(y_t)*(np.outer(1-s, 1-s))
    Aw2_t = -2*normalize(laplacian(Aw2_t), axis=1, norm='l1')
    Ab_t = squareform(y_t)*(np.outer(1-s, s)+np.outer(s, 1-s))
    Ab_t = -2*normalize(laplacian(Ab_t), axis=1, norm='l1')
    y_t = squareform(Y[i])
    s=np.array(membership[i])
    #save
    y.append(y_t)
    Aw.append(Aw_t)
    Aw2.append(Aw2_t)
    Ab.append(Ab_t)

In [ ]:
density = []
for network in Y:
    density.append(np.mean(network))

In [ ]:
density

In [ ]:
R_list = []
D_list = []
for mem in membership:
    R_list.append(np.sum(mem))
    D_list.append(len(mem)-np.sum(mem))

In [ ]:
R_list

In [ ]:
D_list

In [ ]:
# for i in range(1,12):
#     np.save('../compare/name'+str(i)+'.npy', names[i-1])

In [ ]:
persist = np.concatenate(persist)

In [ ]:
label, persist, Aw, Aw2, Ab, combination_N=preprocess_congress_2(y, Aw, Aw2,Ab, n_nodes, persist)

In [ ]:
# Set device for computation (GPU if available)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
# device = "cpu"

In [ ]:
combination_N = combination_N.to(device)
label = label.to(device)
persist = persist.to(device)

In [ ]:
LR = 2e-3
MOM = 0.99
LR_P = 1e-2

# Step 1

In [ ]:
print("Step 1: Fitting initial CLSNA model with higher-dimensional space...")

In [ ]:
#train the model
def train(optimizer,index=None, fixed=None):
    t_index=torch.arange(start=0,end=np.sum(n_nodes),device=device,requires_grad=False)
    optimizer.zero_grad()
    loss = model.loss(device=device,label=label,persist=persist,sample_edge=combination_N,T_index=t_index,ss=SIGMA,tt=TAU,pp=PHI)
    loss.backward()
    model.para.grad = 0.1*((model.para.grad>0).bool().float()-0.5)
    optimizer.step()
    if index is not None:
        with torch.no_grad():
            model.para[index[0],index[1]] = fixed
    return loss.item()

In [ ]:
#run the optimization process
def run(optimizer):
    for epoch in range(1,10000):
        loss = train(optimizer)
        if epoch%300==0:
            print(loss)

In [ ]:
# Initialize and train the first model
model = ClsnaModelCongress(device,n_nodes,T,ar_pair,Aw,Aw2,Ab,new_at_t,member_at_t,D=DIM+1).to(device)

In [ ]:
optimizer = torch.optim.SGD([
    {'params': model.z, "momentum": 0.99, "lr": 0.02},
    {'params': model.para, "momentum": 0, "lr":LR_P}
    ])

In [ ]:
run(optimizer)

In [ ]:
# Perform PCA to reduce dimensionality
PCA_p = torch.pca_lowrank(model.z.cpu())[2][:,[0,1]]
zz=(model.z.cpu().detach()@PCA_p).detach().numpy()
init_z = zz
init_para = model.para.detach().cpu().numpy()

In [ ]:
zz = np.clip(zz, -15, 15)

# Step 2

In [ ]:
print("Step 2: Fitting CLSNA model with targeted dimension and estimating model parameters...")

In [ ]:
model = ClsnaModelCongress(device,n_nodes,T,ar_pair,Aw,Aw2,Ab,new_at_t,member_at_t,D=DIM).to(device)
with torch.no_grad():       
    model.z[:,:] = torch.from_numpy(init_z).to(device)
    model.para[:,:] = torch.from_numpy(init_para).to(device)
optimizer = torch.optim.SGD([
    {'params': model.z, "momentum": 0.99, "lr": 0.02},
    {'params': model.para, "momentum": 0.0, "lr":LR_P}
    ])

In [ ]:
def run(optimizer):
    for epoch in range(1,10000):
#         optimizer.param_groups[0]['lr'] = 0.0005
        loss = train(optimizer)
        if epoch%300==0:
            print(loss)
#             print(model.para)           

In [ ]:
run(optimizer)

In [ ]:
optimizer = torch.optim.SGD([
    {'params': model.z, "momentum": 0.99, "lr": 0.02/2},
    {'params': model.para, "momentum": 0.0, "lr":LR_P/2}
    ])

In [ ]:
run(optimizer)

In [ ]:
zz = model.z.cpu().detach().numpy()

In [ ]:
np.concatenate(membership)

In [ ]:
fig, axes = plt.subplots(2, 3, figsize=(10, 6))  # Create a 2x3 grid of subplots

for i, ax in enumerate(axes.flat):  # Iterate over each subplot
    # Map values to their corresponding slices
    start = int(np.sum(n_nodes[:i*2]))
    end = int(np.sum(n_nodes[:(i+1)*2]))
    
    z = zz[start:end]
    mem_long = np.concatenate(membership)[start:end]
    
    # Use different markers and colors for clarity in grayscale
    dem_positions = z[mem_long == 0]
    rep_positions = z[mem_long == 1]
    
    dem_scatter = ax.scatter(dem_positions[:, 0], dem_positions[:, 1], color='blue',marker='o', s=10, label='Democrats')
    rep_scatter = ax.scatter(rep_positions[:, 0], rep_positions[:, 1], color='red', marker='x', s=10, label='Republicans')
    
    # Customize plot
    ax.set_title(f'Year {i*2+2010}')

# fig.legend([dem_scatter, rep_scatter], ['Democrats', 'Republicans'], loc='upper center', ncol=2, frameon=False)

# Adjust spacing between subplots
plt.tight_layout()

# Show plots
plt.show()

In [ ]:
for i in range(T):
    start = int(np.sum(n_nodes[:i]))
    end = int(np.sum(n_nodes[:(i+1)]))
#     visualize(z_hat=zz,z_true=zz,start=start,end=end)
    visualize_membership(zz,np.concatenate(membership),start,end,caption=None)

In [ ]:
init_z = zz
init_para = model.para.detach().cpu().numpy()

In [ ]:
# np.save('../compare/z6.npy', init_z)

In [ ]:
init_para

In [ ]:
import math
def compute_combinations_length_list():
    # Compute the length of combination_N for each n in n_nodes
    return [math.comb(n, 2) for n in n_nodes]

def get_combination_indices_range(T):
    if T < 0 or T >= len(n_nodes):
        raise ValueError("T is out of range of n_nodes indices.")
    combination_lengths = compute_combinations_length_list()
    start_idx = sum(combination_lengths[:T])
    segment_length = combination_lengths[T]
    end_idx = start_idx + segment_length
    return start_idx, end_idx

for T in range(len(n_nodes)):
    start_idx, end_idx = get_combination_indices_range(T)
    segment_combinations = combination_N[start_idx:end_idx]
    
    with torch.no_grad():
        loss = model.calculate_auc(
            device=device,
            label=label[start_idx:end_idx],
            persist=persist[start_idx:end_idx],
            sample_edge=combination_N[start_idx:end_idx]
        )
    print(f"AUC at time {T}: {loss.item()}")

# Step 3

In [ ]:
print("Step 3: Performing variance/covariance estimation for the parameters of interest...")

In [ ]:
def run(optimizer,index,fixed):
    for epoch in range(1,3000):
        loss = train(optimizer=optimizer,index=index,fixed=fixed)
        if epoch%500 == 0:
            print(loss)
    return loss

In [ ]:
model = ClsnaModelCongress(device,n_nodes,T,ar_pair,Aw,Aw2,Ab,new_at_t,member_at_t,D=DIM).to(device)
with torch.no_grad():       
    model.z[:,:] = torch.from_numpy(init_z).to(device)
    model.para[:,:] = torch.from_numpy(init_para).to(device)
optimizer = torch.optim.SGD([
    {'params': model.z, "momentum": 0.99, "lr": 0.02},
    {'params': model.para, "momentum": 0.0, "lr":LR_P}
    ])    
logL = train(optimizer)

In [ ]:
delta_var = 0.1

In [ ]:
# Estimate variance/covariance for each parameter
parad = {'alpha':(0,1),'delta':(2,1),'gw':(1,1),'gw2':(3,0),'gb':(2,0)}
var_list = {'alpha':(0,1),'delta':(2,1),'gw':(1,1),'gw2':(3,0),'gb':(2,0)}
cov_list = []

for key, value in parad.items():
    model = ClsnaModelCongress(device,n_nodes,T,ar_pair,Aw,Aw2,Ab,new_at_t,member_at_t,D=DIM).to(device)
    with torch.no_grad():       
        model.z[:,:] = torch.from_numpy(init_z).to(device)
        model.para[:,:] = torch.from_numpy(init_para).to(device)
    optimizer = torch.optim.SGD([
    {'params': model.z, "momentum": 0.99, "lr": 0.02},
    {'params': model.para, "momentum": 0.0, "lr":LR_P}
    ])
    newlogL=run(optimizer,value,init_para[value[0],value[1]]+delta_var)
    var_hat = delta_var/(newlogL-logL)**0.5/2**0.5
    var_list[key] = (round(var_hat,5))
    print(key," sd: ", round(var_hat,5))
    diff = model.para-torch.from_numpy(init_para).to(device)
    extracted_values = [diff[value[0], value[1]].item() for value in parad.values()]
    extracted_values = np.array(extracted_values)
    cov_list.append(extracted_values/delta_var*var_hat**2)

In [ ]:
print("sd estimates: ", var_list)

In [ ]:
init_para = init_para.round(3)
printdict = {'a':init_para[0,1],'d':init_para[2,1],'gw':init_para[1,1],'gw2':init_para[3,0],'gb':init_para[2,0]}
print("point estimates: ", printdict)

In [ ]:
with open('estvar', 'a') as file:
    # Convert dictionary to string and write it to the file
    file.write(str(var_list) + '\n')

In [ ]:
with open('est', 'a') as file:
    # Convert dictionary to string and write it to the file
    file.write(str(printdict) + '\n')

In [ ]:
# import csv   
# fields=var_list
# with open('var001', 'a') as f:
#     writer = csv.writer(f)
#     writer.writerow(fields)

In [ ]:
# import csv   
# fields=list(printdict.values())
# with open('theta001', 'a') as f:
#     writer = csv.writer(f)
#     writer.writerow(fields)